# Segmenting and Clustering Data : Part 2

In [13]:
# Install Geocoder
!pip install geocoder

    100% |████████████████████████████████| 102kB 6.7MB/s ta 0:00:01
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Pyt

In [14]:
# Import the required Libraries
import pandas as pd
import numpy as np
import geocoder
print('Imported Libraries!')

Imported Libraries!


We have the Datafram from part 1 stored in a .csv file called Canada_Postcodes.csv. Now, lets import it again and add the Lat/Lon coordinates using the Geocoder Python package.

In [15]:
#Read Dataframe from part 1 "Canada_Postcodes"
df_neighborhoods = pd.read_csv('Canada_Postcodes.csv',index_col=[0])
df_neighborhoods.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
#Lets create a function to get coordinates of particular postcode. Add "Max Iterations" parameter to limit the number of
#iterations, and a "Debug" parameter to show the progress of the for loop

def get_coordinates_Canada(postal_code, max_iterations, debug):
    lat_lng_coords = None
    success=False
    for i in range(max_iterations):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        if debug:
            print('Attempt #: {}, Coordinates: {}'.format(i+1, lat_lng_coords))
        if lat_lng_coords is not None:
            success=True
            break
    if success:
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
        return latitude, longitude
    else:
        return 'Failed to get coordinates'

In [17]:
#Test the function (15 iterations)
get_coordinates_Canada('M1C',15,False)

'Failed to get coordinates'

In [19]:
# Test the Geocoder API
g = geocoder.google('Mountain View, CA')
print(g.latlng)

None


The Geocoder API is not working at the moment. We will use the data in the following link given in the Project Assignment that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [20]:
#Read CSV file and load it into our dataframe
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
# Make sure both dataframes have the same Headings "PostalCode"
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_neighborhoods.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

In [22]:
# Merge both dataframes
df_neighborhoods_coordinates = pd.merge(df_neighborhoods, df_coordinates, on='PostalCode')
df_neighborhoods_coordinates.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [23]:
# Recheck the Coordinates with "PostalCode"
df_neighborhoods_coordinates[(df_neighborhoods_coordinates['PostalCode']=='M7A') |
                             (df_neighborhoods_coordinates['PostalCode']=='M4X') |
                             (df_neighborhoods_coordinates['PostalCode']=='M1H') ]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
51,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
85,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [24]:
#Export to .CSV file
df_neighborhoods_coordinates.to_csv('Canada_Postcodes_2.csv')
print("File Saved")

File Saved
